<a href="https://colab.research.google.com/github/NavyaSri-2001/SQL_Projects/blob/main/SQL_Murder_Mystery_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing libraries
import pandas as pd
import sqlite3 as sqlite3

In [3]:
#setting a connection
con=sqlite3.connect("/content/sql-murder-mystery.db")

In [6]:
#setting DataFrame with to max for better view
pd.set_option('display.max_colwidth', None)

In [9]:
#Viewing the Database schema
query1= '''
SELECT *
FROM sqlite_master
WHERE type='table';
'''
pd.read_sql_query(query1, con)

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,"CREATE TABLE crime_scene_report (\n date integer,\n type text,\n description text,\n city text\n )"
1,table,drivers_license,drivers_license,3,"CREATE TABLE drivers_license (\n id integer PRIMARY KEY,\n age integer,\n height integer,\n eye_color text,\n hair_color text,\n gender text,\n plate_number text,\n car_make text,\n car_model text\n )"
2,table,person,person,4,"CREATE TABLE person (\n id integer PRIMARY KEY,\n name text,\n license_id integer,\n address_number integer,\n address_street_name text,\n ssn integer,\n FOREIGN KEY (license_id) REFERENCES drivers_license(id)\n )"
3,table,facebook_event_checkin,facebook_event_checkin,5,"CREATE TABLE facebook_event_checkin (\n person_id integer,\n event_id integer,\n event_name text,\n date integer,\n FOREIGN KEY (person_id) REFERENCES person(id)\n )"
4,table,interview,interview,6,"CREATE TABLE interview (\n person_id integer,\n transcript text,\n FOREIGN KEY (person_id) REFERENCES person(id)\n )"
5,table,get_fit_now_member,get_fit_now_member,7,"CREATE TABLE get_fit_now_member (\n id text PRIMARY KEY,\n person_id integer,\n name text,\n membership_start_date integer,\n membership_status text,\n FOREIGN KEY (person_id) REFERENCES person(id)\n )"
6,table,get_fit_now_check_in,get_fit_now_check_in,9,"CREATE TABLE get_fit_now_check_in (\n membership_id text,\n check_in_date integer,\n check_in_time integer,\n check_out_time integer,\n FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id)\n )"
7,table,income,income,10,"CREATE TABLE income (\n ssn integer PRIMARY KEY,\n annual_income integer\n )"
8,table,solution,solution,11,"CREATE TABLE solution (\n user integer,\n value text\n )"


In [15]:
#Based on the available info viewing crime scene report of murder
query2='''
SELECT *
FROM CRIME_SCENE_REPORT
WHERE CITY='SQL City'
AND DATE=20180115
'''
pd.read_sql_query(query2,con)

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [18]:
#Viewing schema of person table
query3='''
SELECT sql
FROM sqlite_schema
WHERE NAME='person'
'''
pd.read_sql_query(query3,con)

,sql
0,"CREATE TABLE person (\n id integer PRIMARY KEY,\n name text,\n license_id integer,\n address_number integer,\n address_street_name text,\n ssn integer,\n FOREIGN KEY (license_id) REFERENCES drivers_license(id)\n )"


In [21]:
#viewing info of first witness from person table
query4='''
SELECT *
FROM PERSON
WHERE address_street_name ='Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
'''
pd.read_sql_query(query4,con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [24]:
#identifying second witness
query5='''
SELECT *
FROM PERSON
WHERE address_street_name='Franklin Ave'
AND name LIKE '%Annabel%'
'''
pd.read_sql_query(query5,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [7]:
#getting witness testimonies
query6='''
SELECT *
FROM interview i
JOIN person p ON
p.id=i.person_id
WHERE p.id IN (14887,16371)
'''
pd.read_sql_query(query6,con)

,person_id,transcript,id,name,license_id,address_number,address_street_name,ssn
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [8]:
#Find people whose membership id and check-in date at the gym
#matches witness testimonies.
query7='''
SELECT *
FROM get_fit_now_member gm
JOIN get_fit_now_check_in gc
ON gm.id=gc.membership_id
WHERE gc.membership_id LIKE "48Z%"
AND gc.check_in_date=20180109
'''
pd.read_sql_query(query7,con)

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


In [11]:
#Finding culprit based on Car number
query8='''
SELECT *
FROM person p
join drivers_license d
ON p.license_id=d.id
WHERE p.id IN (28819, 67318)
AND d.plate_number LIKE "%H42W%"
'''
pd.read_sql_query(query8,con)

,id,name,license_id,address_number,address_street_name,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


In [12]:
#Jeremy Bowers Transcript
query9='''
SELECT *
FROM interview i
JOIN person p
ON p.id=i.person_id
WHERE p.id=67318
'''
pd.read_sql_query(query9,con)

,person_id,transcript,id,name,license_id,address_number,address_street_name,ssn
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n",67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


We can search for the physical characterists of the suspect and their car with the drivers_license table and join the facebook_event_checkin table to confirm their concert activity.


In [16]:
#Finding the main culprit
query10='''
SELECT *
FROM drivers_license d
JOIN person p
ON d.id=p.license_id
JOIN facebook_event_checkin f
ON p.id=f.person_id
WHERE d.gender='female' AND d.hair_color='red'
AND d.car_make='Tesla' AND d.car_model='Model S'
'''
pd.read_sql_query(query10,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date
0,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171206
1,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171212
2,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171229


**Based on the above result, Miranda Priestly is the main culprit**